In [28]:
import openai
import pandas as pd
import urllib.request, json 
import string, re
import random
import numpy as np

with open('API_key.txt', 'r') as file:
    openai.api_key = file.readline().strip()

In [76]:
prompt = "Can you answer the following question by answering two subquestions: {}"
bamboogle_df = pd.read_csv('data/bamboogle.csv')
questions = list(bamboogle_df['Question'])
answers = list(bamboogle_df['Answer'])

In [35]:
answers_raw = []
for q in questions:
    ans = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        max_tokens=450,
        stop='',
        messages=[
            {"role": "system", "content": "You are a helpful language model being evaluated for your ability to answer challenging questions. Answer in the format provided by the user."},
            {"role": "user", "content": prompt.format(q)}
        ]
    )
    answers_raw.append(ans['choices'][0]["message"]["content"])
    np.save("./temp/raw.npy", np.array(answers_raw))

In [52]:
with open(r'./temp/temp.txt', 'w') as fp:
    for i, item in enumerate(answers_raw):
        fp.write(f"\n--{i}--\n{item}")


In [80]:
data = []
with open(r'./data/bamboogle_manual_clean.txt', 'r') as file:
    for line in file:
        assert line.rstrip().startswith('--')
        obj = {}
        obj['Question'] = questions[int(line[2:-3])]
        obj['Answer'] = answers[int(line[2:-3])]
        obj['Q1'] = next(file).rstrip()[4:]
        obj['A1'] = next(file).rstrip().lower().split('||')
        next(file)
        obj['Q2'] = next(file).rstrip()[4:]
        obj['A2'] = next(file).rstrip().lower().split('||')
        next(file)
        data.append(obj)


In [86]:
with open('data/bamboogle.json', 'w') as f:
    json.dump(data, f)